# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель**: обучить модель классифицировать комментарии на позитивные и негативные, значением метрики качества *F1* не должно быть  меньше 0.75. В моём распоряжении набор данных с разметкой о токсичности правок.

**План:**

1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Формирование выводов.

**Установка библиотек**

In [1]:
!pip install catboost -q


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install lightgbm -q


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install imblearn -q


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. Подготовка данных

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from imblearn.pipeline import make_pipeline

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import make_scorer, f1_score

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 
import re

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pivka\AppData\Roaming\nltk_data...


True

In [4]:
data = pd.read_csv('toxic_comments.csv', index_col='Unnamed: 0')
data.index.name = 'Number'
data.head()
data = data.reset_index(drop=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Итак, имеем корпус с текстами. *Toxic* - целевой признак, указывает на токчисночть текста. В корпусе 159292 записи.

Создадим функции для предобработки. *lemmatization* - лемматизация, *clear_text* - очистка от лишних символов

In [6]:
def lemmatization(text):
    m = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([m.lemmatize(w) for w in word_list])
    return lemmatized_output


def clear_text(text):
    text = " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split()).lower().replace('/n', ' ')
    return text

Очищаем тексты от лишних символов, оставляем только латиницу и пробелы:

In [7]:
data['text'] = data['text'].apply(lambda x: clear_text(x))

In [8]:
data['text'] = data['text'].apply(lemmatization)

Делим данные на тренировочную и тестовую выборки:

In [9]:
target = data.toxic
features = data.text

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.5, random_state=28)

Создаём TF-IDF матрицы для обучающей и тестовой выборок:

In [10]:
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2. Обучение моделей 

Для подбора будем использовать пайплайн. Тестировать будем случайный лес, CatBoost, логистическую регрессию

In [11]:
pipeline = make_pipeline(TfidfVectorizer(stop_words=stopwords))

In [12]:
pipeline.steps.append(('classification', DummyClassifier()))

In [13]:
def create_randomized_search_cv(pipeline, parameters=None):
    if parameters is None:
        params = [
            {
                'classification': [RandomForestClassifier()],
                'classification__n_estimators': [100,200,300],
                'classification__max_depth': [5, 10]
            },
            {
                'classification': [CatBoostClassifier()],
                'classification__n_estimators': [100,200,300],
                'classification__max_depth': [5, 10]
            },
            {
                'classification': [LogisticRegression()],
                'classification__solver': ['newton-cg', 'lbfgs', 'liblinear'], 
                'classification__C': [0.1, 1, 10]
            },
        ]
    else:
        params = parameters

    grid = RandomizedSearchCV(pipeline,
                              params,
                              cv = 3,
                              verbose = 4,
                              scoring = make_scorer(f1_score),
                              n_jobs=-1)

    return grid

In [14]:
grid = create_randomized_search_cv(pipeline)

In [15]:
%%time

grid.fit(features_train, target_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Learning rate set to 0.464022
0:	learn: 0.3673511	total: 1.02s	remaining: 1m 41s
1:	learn: 0.2754239	total: 1.79s	remaining: 1m 28s
2:	learn: 0.2462914	total: 2.55s	remaining: 1m 22s
3:	learn: 0.2318794	total: 3.31s	remaining: 1m 19s
4:	learn: 0.2225983	total: 4.05s	remaining: 1m 16s
5:	learn: 0.2128780	total: 4.8s	remaining: 1m 15s
6:	learn: 0.2077065	total: 5.57s	remaining: 1m 14s
7:	learn: 0.2032014	total: 6.32s	remaining: 1m 12s
8:	learn: 0.1995650	total: 7.08s	remaining: 1m 11s
9:	learn: 0.1947951	total: 7.89s	remaining: 1m 11s
10:	learn: 0.1920950	total: 8.64s	remaining: 1m 9s
11:	learn: 0.1884770	total: 9.38s	remaining: 1m 8s
12:	learn: 0.1862529	total: 10.1s	remaining: 1m 7s
13:	learn: 0.1838836	total: 10.8s	remaining: 1m 6s
14:	learn: 0.1822837	total: 11.5s	remaining: 1m 5s
15:	learn: 0.1789986	total: 12.2s	remaining: 1m 4s
16:	learn: 0.1770437	total: 12.9s	remaining: 1m 3s
17:	learn: 0.1748369	total: 13.7s	remaining

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('tfidfvectorizer',
                                              TfidfVectorizer(stop_words=['i',
                                                                          'me',
                                                                          'my',
                                                                          'myself',
                                                                          'we',
                                                                          'our',
                                                                          'ours',
                                                                          'ourselves',
                                                                          'you',
                                                                          "you're",
                                                                          "you've",
                                                                          "you'll",
                                                                          "you'd",
                                                                          'your',
                                                                          'yours',
                                                                          'yourself',
                                                                          'yourselves',
                                                                          'he',
                                                                          'him',
                                                                          'his',
                                                                          'himself',
                                                                          'she',
                                                                          "she's",
                                                                          'her',
                                                                          'hers',
                                                                          'herself',
                                                                          'it',
                                                                          "it's",
                                                                          'its',
                                                                          'itself', ...])),
                                             ('classification',
                                              Dumm...
                                        {'classification': [<catboost.core.CatBoostClassifier object at 0x7fd044c4bbe0>],
                                         'classification__max_depth': [5, 10],
                                         'classification__n_estimators': [100,
                                                                          200,
                                                                          300]},
                                        {'classification': [LogisticRegression(C=10,
                                                                               solver='liblinear')],
                                         'classification__C': [0.1, 1, 10],
                                         'classification__solver': ['newton-cg',
                                                                    'lbfgs',
                                                                    'liblinear']}],
                   scoring=make_scorer(f1_score), verbose=4)

Лучшая модель:

In [19]:
grid.best_params_

{'classification__solver': 'liblinear',
 'classification__C': 10,
 'classification': LogisticRegression(C=10, solver='liblinear')}

Результат на обучающей выборке:

In [20]:
grid.best_score_

0.7524999022892541

Предсказываем на тестовой выборке:

In [21]:
predictions = grid.predict(features_test)

Полученный результат метрики на тестовой выборке:

In [22]:
f1_score(target_test, predictions)

0.7640906857584053

**Вывод**

Наилучшим образом показала себя модель линейной регрессии, достигнув значения 0.76 на тестовой выборке. В целом, классификация текстов - очень интересный и трудный процесс, требующий много вычислительных мощностей. Большое влияние на конечный результат оказывает предобработка.